# Login HuggingFace to Upload Model

In [1]:
# !git config --global credential.helper store

# from huggingface_hub import notebook_login
# notebook_login()

# 1. Training

In [2]:
import torch
torch.cuda.empty_cache()

In [5]:
!nvidia-smi

Tue Mar  8 02:18:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   46C    P0    41W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A   47C    P0    40W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [6]:
def print_encoded(encoded_inputs):
    for ts in encoded_inputs:
        print(ts, encoded_inputs[ts])

## 1.1 Prepare-Datasets

In [7]:
import transformers
print(transformers.__version__)

4.17.0


In [8]:
model_checkpoint = "Helsinki-NLP/opus-mt-th-en"

In [9]:
from datasets import load_dataset, load_metric, DatasetDict, Dataset

# raw_datasets = load_dataset("scb_mt_enth_2020", name="then")
metric = load_metric("sacrebleu")

Downloading:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

In [17]:
import pandas as pd
def pre_process_from_csv(path, n_row=100000):
    df_5M = pd.read_csv(path, nrows=n_row)
    list_5M = df_5M.to_dict('records')
    list_sub = ['LST_Corpus']*len(list_5M)
    dict_5M = pd.DataFrame({"translation": list_5M, "subdataset": list_sub})
    return dict_5M

In [23]:
raw_datasets = pre_process_from_csv('TEST_dataset_5M.csv', 500000)

In [24]:
raw_datasets['translation']

0         {'th': 'ขอบคุณ แต่ ฉัน ไม่ มี เวลา พอ', 'en': ...
1         {'th': 'โอ้ , คุณ รู้ , เมื่อ ฉัน ตื่น ฉัน จะ ...
2         {'th': 'หน้าที่ ของ ตำรวจ คือ การ ปกป้อง ประชา...
3         {'th': 'มัน เป็นความ โอหัง ของ เพศ หญิง .', 'e...
4         {'th': 'โปรด คอย ดู อย่า เผลอ', 'en': 'please ...
                                ...                        
499995    {'th': 'แม่ ที่ ให้ ลูก กิน นม ของ ตน เอง', 'e...
499996    {'th': 'เขาอ ยู่ ใน สาย ตอน นี้', 'en': 'he is...
499997                  {'th': 'ตรงกลาง', 'en': 'centered'}
499998    {'th': 'ทิศทาง การ จัด วาง', 'en': 'orientation'}
499999    {'th': 'ล็อค พวง กุญแจ ของ G NO M E ขณะ เครื่อ...
Name: translation, Length: 500000, dtype: object

In [29]:
cut_datasets = DatasetDict()
cut_datasets = Dataset.from_pandas(raw_datasets, split="train+validation").train_test_split(0.1)

In [30]:
cut_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 450000
    })
    test: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 50000
    })
})

In [31]:
import datasets, random
import pandas as pd
from IPython.display import display, HTML

def show_random_element(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than dataset"
    picks = []
    for n_ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks: # random inside
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [32]:
show_random_element(cut_datasets['train'])

,translation,subdataset
0,"{'en': 'the keeper of the livery-stable pressed sombat hard , trying to sell him his trotting horse .', 'th': 'คน ดูแล คอก ม้า โน้มน้าว คุณสมบัติ อย่าง หนัก โดย พยายาม จะ ขาย ม้า ตัว ที่ วิ่ง เหยาะแหยะ ให้ เขา'}",LST_Corpus
1,"{'en': 'preserve paintings from deterioration', 'th': 'รักษา ภาพ จิตรกรรม ไว้ มิ ให้ เสื่อม สภาพ'}",LST_Corpus
2,"{'en': 'call the airline.', 'th': 'โทร หา สายการบิน .'}",LST_Corpus
3,"{'en': 'harare kutsaga', 'th': 'ฮา รา เ ร Ku t s a g a'}",LST_Corpus
4,"{'en': 'they were just like any other young couple .', 'th': 'พวก เขา ก็ เหมือนกับ คู่ หนุ่มสาว อื่นๆ'}",LST_Corpus


In [33]:
cut_datasets['train'][0:2]

{'translation': [{'en': 'they formed a circle and sang .',
   'th': 'พวก เขา ยืน ล้อมวง แล้ว ร้อง เพลง'},
  {'en': 'it was a tearjerker.', 'th': 'มัน เป็น เรื่อง เศร้า .'}],
 'subdataset': ['LST_Corpus', 'LST_Corpus']}

#### Metric

In [34]:
# metric

In [35]:
metric = load_metric("sacrebleu")
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

## 1.2 Preprocessing the Data

In [36]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [37]:
# Set Source and Target Language

if 'mbart' in model_checkpoint:
    tokenizer.src_lang = 'th'
    tokenizer.tgt_lang = 'en'

In [38]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    model_checkpoint = "translate Thai to English: "
else:
    prefix = ""

In [103]:
max_input_len  = 128
max_target_len = 128

source_lang = 'th'
target_lang = 'en'

def preprocess_function(examples):
#     inputs = [prefix + ex[source_lang] for ex in examples['translation']]
    inputs, targets = [], []
    for ex_ in (examples['translation']):
        ex_target = ex_[target_lang]
        if ex_target is not None:
            targets.append(ex_[target_lang])
            inputs.append(prefix + ex_[source_lang])
            
#     if len(targets) != len(inputs):
#         print(inputs)
#         print()
#         print(targets)

    model_inputs = tokenizer(inputs, max_length=max_input_len, truncation=True) # Pad to longest word (128 char)
#     print(model_inputs['input_ids'])
    
    with tokenizer.as_target_tokenizer():
        try:
            labels = tokenizer(targets, max_length=max_target_len, truncation=True)
        except:
            print('targets: ', targets)
    
    
#     if len(model_inputs['input_ids']) != len(model_inputs['input_ids']):
#         print(model_inputs)
        
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [108]:
max_input_len  = 128
max_target_len = 128

source_lang = 'th'
target_lang = 'en'

def preprocess_function(examples):
#     inputs = [prefix + ex[source_lang] for ex in examples['translation']]
    inputs, targets = [], []
    for ex_ in (examples['translation']):
        ex_target = ex_[target_lang]
        if ex_target is not None:
            targets.append(ex_[target_lang])
            inputs.append(prefix + ex_[source_lang])
            
    model_inputs = tokenizer(inputs, max_length=max_input_len, truncation=True) # Pad to longest word (128 char)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_len, truncation=True)
    
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [115]:
tokenized_datasets = cut_datasets.map(preprocess_function, 
                                      batched=True, 
                                      batch_size=1000, 
                                      num_proc=32, 
                                      remove_columns=["translation", "subdataset"], # Prevent ArrowInvalid Error'
                                     )

In [117]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 449999
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 50000
    })
})

## 1.3 Fine-tuning the model

In [118]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [119]:
model_checkpoint

'Helsinki-NLP/opus-mt-th-en'

In [120]:
torch.cuda.empty_cache()

In [122]:
!nvidia-smi

Tue Mar  8 03:00:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   43C    P0    40W / 300W |     11MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A   45C    P0    39W / 300W |     11MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [128]:
batch_size = 32
model_name = model_checkpoint.split('/')[-1]
args = Seq2SeqTrainingArguments(
    f'{model_name}-finetuned-5k-{source_lang}-to-{target_lang}',
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size//2,
#     gradient_checkpointing=True, # For Accelerate Training
    weight_decay =0.01,
    save_total_limit = 3,
    num_train_epochs=1,
    predict_with_generate=True,
    dataloader_num_workers=32, # Multi-tread CPU
    fp16=True,
    eval_accumulation_steps = 10, # Reduce Using GPU Ram When Evaulation
    push_to_hub = False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
?Seq2SeqTrainingArguments

#### Create Data collator

In [130]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [131]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels= [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    print('preds', preds)
    print('labels', labels)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens = True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result['score']}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [132]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset= tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test'],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics,
)

Using amp half precision backend


## 1.4 *train time!*

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 449999
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 3516
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


Saving model checkpoint to opus-mt-th-en-finetuned-5k-th-to-en/checkpoint-500
Configuration saved in opus-mt-th-en-finetuned-5k-th-to-en/checkpoint-500/config.json
Model weights saved in opus-mt-th-en-finetuned-5k-th-to-en/checkpoint-500/pytorch_model.bin
tokenizer config file saved in opus-mt-th-en-finetuned-5k-th-to-en/checkpoint-500/tokenizer_config.json
Special tokens file saved in opus-mt-th-en-finetuned-5k-th-to-en/checkpoint-500/special_tokens_map.json
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [106]:
?trainer.push_to_hub

Signature:
trainer.push_to_hub(
    commit_message: Union[str, NoneType] = 'End of training',
    blocking: bool = True,
    **kwargs,
) -> str
Docstring:
Upload *self.model* and *self.tokenizer* to the 🤗 model hub on the repo *self.args.hub_model_id*.

Parameters:
    commit_message (`str`, *optional*, defaults to `"End of training"`):
        Message to commit while pushing.
    blocking (`bool`, *optional*, defaults to `True`):
        Whether the function should return only when the `git push` has finished.
    kwargs:
        Additional keyword arguments passed along to [`~Trainer.create_model_card`].

Returns:
    The url of the commit of your model in the given repository if `blocking=False`, a tuple with the url of
    the commit and an object to track the progress of the commit if `blocking=True`
File:      /usr/local/lib/python3.8/dist-packages/transformers/trainer.py
Type:      method


In [107]:
!git lfs install
trainer.push_to_hub()

Updated Git hooks.
Git LFS initialized.


OSError: Tried to clone a repository in a non-empty folder that isn't a git repository. If you really want to do this, do it manually:
git init && git remote add origin && git pull origin main
 or clone repo to a new folder and move your existing files there afterwards.

In [ ]:
!ls

When Loading the model


```
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```



# 2. Using Translation Model

### Using AutoModel (With model selection)

In [108]:
?AutoModelForSeq2SeqLM.

Object `AutoModelForSeq2SeqLM.` not found.


In [110]:
# metric = load_metric("bleu")
# fake_preds = [["hello"],["there"], ["general"], ['kenobi']]
# fake_labels = [["hello"],["there"], ["general"], ['kenobi']]
# # fake_labels = [["hello there"], ["general kenobi"]]
# metric.compute(predictions=fake_preds, 
#                references=fake_labels)

In [111]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_pretrain = "Helsinki-NLP/opus-mt-th-en"
model_for_predict = "./opus-mt-th-en-finetuned-5k-th-to-en"
# model_for_predict = "huak95/opus-mt-th-en-finetuned-5k-th-to-en"

model_pt = AutoModelForSeq2SeqLM.from_pretrained(model_pretrain)
model = AutoModelForSeq2SeqLM.from_pretrained(model_for_predict)
tokenizer = AutoTokenizer.from_pretrained(model_pretrain)

loading configuration file https://huggingface.co/Helsinki-NLP/opus-mt-th-en/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/0eb33d380c174e6420413d055247f46d08f0270c5b21934639530ea9cccdd756.2812763906fad9e04edf3afcd7e6da1e82df7c1856291e12132b346b18f9ee46
Model config MarianConfig {
  "_name_or_path": "Helsinki-NLP/opus-mt-th-en",
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      62306
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 62306,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "

OSError: Can't load config for './opus-mt-th-en-finetuned-5k-th-to-en'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure './opus-mt-th-en-finetuned-5k-th-to-en' is the correct path to a directory containing a config.json file

In [112]:
!git lfs track "opus-mt-th-en-finetuned-5k-th-to-en/*"

model.push_to_hub('huak95/opus-mt-th-en-finetuned-5k-th-to-en')
# tokenizer.push_to_hub('huak95/opus-mt-th-en-finetuned-5k-th-to-en')

Tracking "opus-mt-th-en-finetuned-5k-th-to-en/*"


/usr/local/lib/python3.8/dist-packages/huggingface_hub/hf_api.py:1001: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  warnings.warn(
Cloning https://huggingface.co/huak95/opus-mt-th-en-finetuned-5k-th-to-en into local empty directory.


Download file pytorch_model.bin:   0%|          | 604/290M [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/290M [00:00<?, ?B/s]

Configuration saved in huak95/opus-mt-th-en-finetuned-5k-th-to-en/config.json
Model weights saved in huak95/opus-mt-th-en-finetuned-5k-th-to-en/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 32.0k/290M [00:00<?, ?B/s]

To https://huggingface.co/huak95/opus-mt-th-en-finetuned-5k-th-to-en
   a487b0f..c3d5cf2  main -> main



'https://huggingface.co/huak95/opus-mt-th-en-finetuned-5k-th-to-en/commit/c3d5cf2c24040a1523e2e6e956db633732643167'

In [113]:
def harry_translate(input_texts, model=model_pt):
    inputs = tokenizer(input_texts['th'], return_tensors = "pt")

    outputs = model.generate(inputs["input_ids"],
                             max_length=40,
                             num_beams=4,
                             early_stopping=True)

    return tokenizer.decode(outputs[0]).replace('<pad> ',''), input_texts['en']

In [114]:
for i in range(10):
    text_ = tokenized_datasets['test']['translation'][i]
    thai_text = text_['th']
    pred_text_pt, real_text = harry_translate(text_, model_pt)
    pred_text, real_text = harry_translate(text_, model)
    
#     m_score = compute_metrics.compute(predictions=[pred_text], references=[real_text])

    print('thai_text:',  thai_text)
    print('real_text: ', real_text)
    print('pred_text: ', pred_text)
    print('pre_train: ', pred_text_pt)
    print()

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)